<a href="https://colab.research.google.com/github/00Alex16/ProyectoFinal/blob/main/Sistema_para_an%C3%A1lisis_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as rq
import re
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import gensim
from gensim import corpora
from gensim.corpora.dictionary import Dictionary
from afinn import Afinn

# Obtener contenido del archivo desde github
raw_url = "https://raw.githubusercontent.com/00Alex16/ProyectoFinal/main/textos/ejemplo_1.txt"
text = rq.get(raw_url).text

# Función para limpiar y tokenizar el texto
def process_doc(document):
    
    new_doc = document.lower()
    new_doc = re.sub('http\S+', ' ', new_doc)
    punctuation_marks = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    new_doc = re.sub(punctuation_marks , ' ', new_doc)
    new_doc = re.sub("\d+", ' ', new_doc)
    new_doc = re.sub("\\s+", ' ', new_doc)
    new_doc = new_doc.split(sep = ' ')
    new_doc = [token for token in new_doc if len(token) > 1]
    
    return(new_doc)

# Guardamos la lista con las palabras relevantes
new_text = process_doc(text)

# Función para establecer similitudes en el texto
def topic_identification(tokens):

  stop_words = list(stopwords.words('english'))
  filtered_words = [word for word in tokens if word not in stop_words]

  l = len(filtered_words)
  doc = [filtered_words[0:int(l/4)], filtered_words[int(l/4):int(2*l/4)], filtered_words[int(2*l/4):int(3*l/4)], filtered_words[int(3*l/4):]]
  dictionary = corpora.Dictionary(doc)
  DT_matrix = [dictionary.doc2bow(d) for d in doc]
  Lda_object = gensim.models.ldamodel.LdaModel
  lda_model_1 = Lda_object(DT_matrix, num_topics=3, id2word = dictionary)
  return lda_model_1.print_topics(num_topics=3, num_words=4)

# Función para identificar los personajes
def characters_identification(tokens):

  tags = nltk.pos_tag(tokens)
  nouns = [(word, tag) for word, tag in tags if tag in ['NN']]
  return nouns

# Función para realizar el análisis de sentimientos
def sent_analysis(tokens):

  analysis = ""
  lemmatizer = WordNetLemmatizer()
  lem_tokens = [lemmatizer.lemmatize(t) for t in tokens]
  lem_text = " ".join(lem_tokens)
  afn = Afinn()
  if afn.score(lem_text) > 5:
    return "Positivo"
  elif afn.score(lem_text) < -5:
    return "Negativo"
  else:
    return "Neutro"